### High level methods

In [1]:
import earthkit.data
from earthkit.data.wrappers.ndarray import NumpyNDArrayWrapper
from earthkit.meteo import solar

import numpy as np

class Accessor:
    def __init__(self, name, data):
        self.name = name
        self.data = data

    @staticmethod
    def make(name, data):
        a = accessors.get(name, None)
        if a is None:
            a = ParamAccessor

        return a(name, data)

    def is_numpy(self):
        return isinstance(self.data, NumpyNDArrayWrapper)
    
class DateAccessor(Accessor):
    def get(self):
        return self.data.metadata("valid_datetime")
        
        
class LatAccessor(Accessor):
    def get(self):               
        if self.is_numpy():
            return self.data.to_numpy()
        return self.data.to_latlon()["lat"]

class LonAccessor(Accessor):
    def get(self):
        if self.is_numpy():
            return self.data.to_numpy()
        return self.data.to_latlon()["lon"]

class ParamAccessor(Accessor):
    def get(self):
        if self.is_numpy():
            return self.data.to_numpy()
        return self.data.sel(param=self.name).to_numpy()


class PressureAccessor(Accessor):
    def get(self):
        if self.is_numpy():
            return self.data.to_numpy()
        return np.asarray(self.data.metadata("level"))*100.
    

accessors = {"latitude": LatAccessor, "longitude": LonAccessor, 
             "date": DateAccessor, "p": PressureAccessor}

def param_args(*argX):
    def inner_decorator(func):
        def wrapped(*args, **kwargs):
            m_args = []
            for x in argX:
                if isinstance(x, list):
                    m_args.extend(x)
                else:
                    m_args.append(x)
             
            params = []

            w = None
            for i, x in enumerate(m_args):
                if x is None:
                    a = args[i]
                else:
                    if w is None:
                        w = args[i] 
                        w = earthkit.data.from_object(w)
                    ac = Accessor.make(x, w)
                    a = ac.get()
                params.append(a)

            #print(f"{params=}")
            return func(*params, *kwargs)
            
        return wrapped
    return inner_decorator

In [2]:
earthkit.data.download_example_file("tuv_pl.grib")
ds = earthkit.data.from_source("file", "tuv_pl.grib")

### cos_solar_zenith_angle

The array version of **solar.cos_solar_zenith_angle** is defined as follows:

  def cos_solar_zenith_angle(date, latitudes, longitudes)

In [3]:
# The high level version
@param_args("date", "latitude", "longitude") 
def cos_solar_zenith_angle(*args, **kwargs):
    res = solar.cos_solar_zenith_angle(*args, **kwargs)
    return res

In [4]:
# get input values from first field
dv = ds[0].metadata("valid_datetime")
latv = ds[0].to_latlon(flatten=True)["lat"][0]
lonv = ds[0].to_latlon(flatten=True)["lon"][0]
dv, latv, lonv

(datetime.datetime(2018, 8, 1, 12, 0), 90.0, 0.0)

In [5]:
# call array version with numbers to get reference
solar.cos_solar_zenith_angle(dv, latv, lonv)

0.3112577791163394

In [6]:
# calls high level version with a field
res = cos_solar_zenith_angle(ds[0])

In [7]:
res.shape

(7, 12)

In [8]:
res[0, 0]

0.3112577791163394

### potential_temperature

This method is not yet adedd to earthkit meteo so we use a local version.

In [9]:
# the array version
def _potential_temperature(t, p):
    # t: temperature in K
    # p: pressure in Pa
    return t*(100000./p)**0.285611

The main problem in writing the decorator is that "p" should be only extracted to for the fields containing "t". This very simple syntax is not able to do that. Calling this method with the example fieldlist fails, since there are 6 "t" fields but 18 pressure level fields (so 18 pressures) in total.

In [10]:
@param_args("t", "p") 
def potential_temperature(*args, **kwargs):
    res = _potential_temperature(*args, **kwargs)
    return res

In [11]:
# call array version with numbers to get reference
_potential_temperature(264., 85000.)

276.54299249939953

In [12]:
# calls high level version with a field
res = potential_temperature(ds)

ValueError: operands could not be broadcast together with shapes (6,7,12) (18,) 